In [ ]:


# ⚙️ Dla zmiennych numerycznych – mediana
imputer_median = SimpleImputer(strategy="median")
df[num_cols] = imputer_median.fit_transform(df[num_cols])

# ⚙️ Dla zmiennych kategorycznych – najczęstsza wartość
imputer_mode = SimpleImputer(strategy="most_frequent")
df[cat_cols] = imputer_mode.fit_transform(df[cat_cols])

# Dodatkowo: jeśli jakieś kolumny miały braki, można dodać flagi „brak_danych”
for c in df.columns:
    if df[c].isna().any():
        df[c + "_missing"] = df[c].isna().astype(int)


In [ ]:
# ----------------------------------------------------
# 2.4. Wykrywanie i obsługa obserwacji odstających
# ----------------------------------------------------

def clip_outliers(series, lower_q=0.01, upper_q=0.99):
    lower = series.quantile(lower_q)
    upper = series.quantile(upper_q)
    return series.clip(lower, upper)

for c in num_cols:
    df[c] = clip_outliers(df[c])

print("📏 Dane przycięte do 1%–99% zakresu wartości.")

# ----------------------------------------------------
# 2.5. Rozkłady i zależności z targetem
# ----------------------------------------------------

target_col = "target"

# Histogramy i boxploty
for c in num_cols[:6]:  # ogranicz do kilku na start
    plt.figure(figsize=(8,3))
    sns.histplot(data=df, x=c, hue=target_col, kde=True, element="step")
    plt.title(f"Rozkład {c} vs target")
    plt.show()

# Korelacje między zmiennymi numerycznymi
corr = df[num_cols].corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr, cmap="coolwarm", center=0)
plt.title("Macierz korelacji (numeryczne zmienne)")
plt.show()

# Zmienność i multikolinearność (usuwanie bardzo skorelowanych)
corr_threshold = 0.9
high_corr = [(c1, c2) for c1 in num_cols for c2 in num_cols if c1 != c2 and abs(corr.loc[c1, c2]) > corr_threshold]
if high_corr:
    print("⚠️ Bardzo skorelowane zmienne:")
    for pair in high_corr:
        print(f" - {pair[0]} vs {pair[1]} (corr={corr.loc[pair[0], pair[1]]:.2f})")

# ----------------------------------------------------
# 2.6. Binning i WoE
# ----------------------------------------------------

# Prosty monotoniczny binning (dzielenie na decyle)
def monotonic_binning(df, feature, target, n_bins=10):
    df_temp = df[[feature, target]].copy()
    df_temp["bin"] = pd.qcut(df_temp[feature], q=n_bins, duplicates="drop")
    grouped = df_temp.groupby("bin")[target].agg(["count", "mean"])
    grouped["WoE"] = np.log((1 - grouped["mean"]) / grouped["mean"] + 1e-6)
    return grouped

# Przykład dla jednej zmiennej
example_var = num_cols[0] if len(num_cols) > 0 else None
if example_var:
    print(f"📊 Przykład binningu dla: {example_var}")
    display(monotonic_binning(df, example_var, target_col))

# ----------------------------------------------------
# 2.7. Standaryzacja cech
# ----------------------------------------------------

scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

print("✅ Standaryzacja numerycznych cech zakończona.")